In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append('..')

from backtest.performance_generater import PerformanceGenerator
from backtest.operators import *

In [10]:
open = pd.read_csv('/Volumes/Database/Factor_data/open.csv', index_col=0)
close = pd.read_csv('/Volumes/Database/Factor_data/close.csv', index_col=0)
returns = pd.read_csv('/Volumes/Database/Factor_data/return.csv', index_col=0)

# 要再確認一下 報酬正確性
unadj_return = close.pct_change().shift(-2) 
exp_returns = returns.shift(-2) * 0.01

# operatingmargin = pd.read_csv('/Volumes/Database/Factor_data/營業利益率_90.csv', index_col=0) * 0.01
# pb_q = pd.read_csv('/Volumes/Database/Factor_data/當季季底PB_90.csv', index_col=0)
# revenue_m = pd.read_csv('/Volumes/Database/Factor_data/以合併為主單月營收(千元)_90.csv', index_col=0)
# outstanding = pd.read_csv('/Volumes/Database/Factor_data/流通在外股數(千股).csv', index_col=0)

/var/folders/fx/jn0gghjd4cg68lr15nlpsbqr0000gn/T/ipykernel_5885/579887732.py:6: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



In [11]:
ZTXA_close = pd.read_csv('/Volumes/Database/Factor_data/ZXTA_close.csv', index_col=0)
ZTXA_close.index = pd.to_datetime(ZTXA_close.index)
market_returns = ZTXA_close['ZTXA'].pct_change()
Benchmark = ZTXA_close['ZTXA'].pct_change().shift(-2)

In [15]:
mean_close = ts_mean(close,75)
ADSPC_75 = (mean_close - close.shift(1))/mean_close

In [ ]:
### 若他想要的是2個月半
mean_close = ts_mean(close,52)
ADSPC_75 = (mean_close - close.shift(1))/mean_close

In [16]:
factor = ADSPC_75 

# no fee
returns_by_period, summary_df = PerformanceGenerator(factor = factor,
                                                     expreturn = exp_returns,
                                                     strategy='LO',
                                                     buy_fee = 0, 
                                                     sell_fee = 0,
                                                     benchmark = Benchmark
                                                     ).backtest()

# fee
returns_by_period, summary_df = PerformanceGenerator(factor = factor,
                                                     expreturn = exp_returns,
                                                     strategy='LO',
                                                     benchmark = Benchmark
                                                     ).backtest()

+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+
|             | Cumprod Total Returns | Cumsum Total Returns | Sharpe Ratio | Annualized Ret | Max Drawdown | Volatility |  STD   | Turnover |
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+
| Performance |       339.03 %        |       162.47 %       |     0.94     |    15.35 %     |   36.63 %    |  16.64 %   | 1.05 % | 14.74 %  |
|  Benchmark  |       145.24 %        |       103.31 %       |     0.62     |     9.05 %     |   31.51 %    |  16.17 %   | 1.02 % |   nan    |
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+


+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+
|             | Cumprod Total Returns | Cumsum Total Returns | Sharpe Ratio | Annualized Ret | Max Drawdown | Volatility |  STD   | Turnover |
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+
| Performance |        -2.85 %        |       11.60 %        |     0.07     |    -0.28 %     |   58.61 %    |  16.64 %   | 1.05 % | 14.74 %  |
|  Benchmark  |       145.24 %        |       103.31 %       |     0.62     |     9.05 %     |   31.51 %    |  16.17 %   | 1.02 % |   nan    |
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+


In [14]:
csv_file = '/Volumes/Database/medina_alpha/AvgPriceDeviation_75d.csv'
factor.to_csv(csv_file)